In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
dataset = pd.read_csv('churn_modelling\Churn_modelling.csv')

<>:1: SyntaxWarning: invalid escape sequence '\C'
<>:1: SyntaxWarning: invalid escape sequence '\C'
C:\Users\91993\AppData\Local\Temp\ipykernel_22588\152620033.py:1: SyntaxWarning: invalid escape sequence '\C'
  dataset = pd.read_csv('churn_modelling\Churn_modelling.csv')


In [4]:
X = dataset.iloc[: , 3:13]
y = dataset.iloc[: , 13]

In [5]:
# create dummy variables
geography = pd.get_dummies(X['Geography'],drop_first=True)
gender = pd.get_dummies(X['Gender'] , drop_first=True)

In [6]:
X = pd.concat([X,geography,gender],axis=1)

In [7]:
X= X.drop(['Geography' , 'Gender'] , axis=1)

In [8]:
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(X,y , test_size=0.2 , random_state=0)

In [9]:
# feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [25]:
# Performing the hyperparameter tuning function
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Embedding, Flatten, LeakyReLU, BatchNormalization, Dropout
from tensorflow.keras.activations import relu, sigmoid

In [26]:
def create_model(layers, activation):
    model = Sequential()
    for i, nodes in enumerate(layers):
        if i==0:   # for the first layer we need to specify the input dimensions
            model.add(Dense(nodes,input_dim=X_train.shape[1]))     
            model.add(Activation(activation))   # which axtivation function we have to apply
            model.add(Dropout(0.3))    # if any dropout is required
        else:
            model.add(Dense(nodes))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
            
    model.add(Dense(units = 1, kernel_initializer= 'glorot_uniform', activation = 'sigmoid')) # Note: no activation beyond this point
    
    model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [27]:
model = KerasClassifier(build_fn=create_model, verbose=0)

In [28]:

layers = [[20], [40, 20], [45, 30, 15]]    # this is basically trials thta in first trial we well have only one hidden layer , then in next we will have two hidden layer likewise
activations = ['sigmoid', 'relu']
param_grid = dict(layers=layers, activation=activations, batch_size = [128, 256], epochs=[30])
grid = GridSearchCV(estimator=model, param_grid=param_grid,cv=5)

In [ ]:
grid_result = grid.fit(X_train, y_train)

In [ ]:
print(grid_result.best_score_, grid_result.best_params_)

# it will print which combination of the hyperparameters is giving the best result
# like activation:'relu' , batch_size:128 , epochs:30 , layers:[45,30,15]

pred_y = grid.predict(X_test)
y_pred = (pred_y > 0.5)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

from sklearn.metrics import accuracy_score
score = accuracy_score(y_pred, y_test)
